# Super simple Kubeflow Pipelines

Here we will schedule a one pipeline that will download artifacts from minio buckets

In [1]:
!pip3 install --upgrade './extensions'
!pip install --upgrade 'https://storage.googleapis.com/ml-pipeline/release/0.1.8/kfp.tar.gz'

# jupyter notebook extensions
%load_ext extensions

# imports goes here
from ipython_secrets import get_secret
from os import environ
import kfp
import kfp.compiler as compiler
import boto3
import tarfile

Processing ./extensions


  Found existing installation: extensions 0.0.1
    Uninstalling extensions-0.0.1:
      Successfully uninstalled extensions-0.0.1
  Running setup.py install for extensions ... done
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://storage.googleapis.com/ml-pipeline/release/0.1.8/kfp.tar.gz
  Found existing installation: kfp 0.1
    Uninstalling kfp-0.1:
      Successfully uninstalled kfp-0.1
  Running setup.py install for kfp ... done
You are using pip version 18.1, however version 19.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.



## Environment setup

In [2]:
EXPERIMENT_NAME = 'Das-Experiment-1'

AWS_S3_ENDPOINT = environ.get('AWS_S3_ENDPOINT', 's3.amazonaws.com')
AWS_S3_BUCKET = environ.get('AWS_S3_BUCKET')
AWS_DEFAULT_REGION = environ.get('AWS_DEFAULT_REGION', 'us-east-1')
AWS_ACCESS_KEY_ID = get_secret('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = get_secret('AWS_SECRET_ACCESS_KEY')
AWS_SECRET_NAME = environ.get('AWS_SECRET_NAME')

TAG = 'latest'
DOCKER_REGISTRY = environ.get('DOCKER_REGISTRY', 'docker.io')
DOCKER_IMAGE = f'{DOCKER_REGISTRY}/library/kubectl:{TAG}'

Create API clients for pipelines and object storage

In [3]:
s3_client = boto3.client('s3',
    region_name = AWS_DEFAULT_REGION,
    aws_access_key_id = AWS_ACCESS_KEY_ID,
    aws_secret_access_key = AWS_SECRET_ACCESS_KEY)

client = kfp.Client()

*Create experiment*: we retrieve an experiment by name or otherwise will create new

In [25]:
try:
    experiment = client.get_experiment(experiment_name=EXPERIMENT_NAME)
except:
    experiment = client.create_experiment(EXPERIMENT_NAME)

2019-01-25 11:57:18,906 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109bfcf98>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)': /apis/v1beta1/experiments?page_token=&page_size=100&sort_by=
2019-01-25 11:57:18,906 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109bfcf98>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)': /apis/v1beta1/experiments?page_token=&page_size=100&sort_by=
2019-01-25 11:57:18,906 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109bfcf98>: Failed to establish a new co

2019-01-25 11:57:43,943 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109bfc3c8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)': /apis/v1beta1/experiments
2019-01-25 11:57:43,943 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109bfc3c8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)': /apis/v1beta1/experiments
2019-01-25 11:57:43,943 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109bfc3c8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)': 

MaxRetryError: HTTPConnectionPool(host='ml-pipeline.kubeflow.svc.cluster.local', port=8888): Max retries exceeded with url: /apis/v1beta1/experiments (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x109c0b710>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

# Build an experiment related docker image

Below we generate a dockerfile that will be used to put some `minio` awarness:
- `Dockerfile` - a docker container that will be built and pushed into private docker egistry
- `Kaniko` - deploument job to carry on our container build

In [5]:
%%template Dockerfile
FROM gcr.io/google-samples/ml-pipeline-t2ttrain:latest
RUN echo "{{DOCKER_IMAGE}}"

/Users/akranga/dev/applications/app-templates/kubeflow-pipeline/Dockerfile

In [6]:
%templatefile extensions/templates/kaniko-workflow.yaml -o kaniko.yaml

/Users/akranga/dev/applications/app-templates/kubeflow-pipeline/kaniko.yaml

Generated files must be uploaded to object storage bucket (i.e s3, minio). Docker build process (Kaniko) will have to access to s3 bucket

In [24]:
with tarfile.open('dockerbuild.tar.gz', 'w:gz') as tar:
    tar.add('Dockerfile', arcname='Dockerfile')
    tar.add('kaniko.yaml', arcname='pipeline.yaml')

s3_client.upload_file('dockerbuild.tar.gz' , AWS_S3_BUCKET, f'{EXPERIMENT_NAME}/dockerbuild.tar.gz')

r = client.run_pipeline(expoeriment.id, 'docker build', 'dockerbuild.tar.gz')

In [ ]:
client.wait_for_run_completion(r.id, 180)

# Define an Experiment Pipeline

In [12]:
import kfp.dsl as dsl
from kubernetes import client as k8sc

@dsl.pipeline(
  name='Super simple minio integration',
  description='I as a pipeline want to read a file from minio bucket'
)
def hello_minio_pipeline(filename: dsl.PipelineParam):
    op1 = dsl.ContainerOp(
        name='download',
        image='minio/mc',
        command=['mc', '--no-color'],
        arguments=['cp', f'minio/{filename}', '/tmp/results.txt'],
        file_outputs={'downloaded': '/tmp/results.txt'}
    ).add_env_variable(
        k8sc.V1EnvVar(
            name='MC_HOSTS_minio', 
            value=f'https://{S3_ACCESS_KEY}:{S3_SECRET_KEY}@{S3_ENDPOINT}' 
        ))
    op2 = dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=[f'echo {op1.output}'])

### Execute the pipeline

Code below will create a new experiement **Hello Minio** and run it

In [9]:
compiler.Compiler().compile(hello_minio_pipeline, 'pipeline.tar.gz')
run = client.run_pipeline(exp.id, 
                          'pipeline 1', 
                          'pipeline.tar.gz',
                          params={'filename': 'default/hello.txt'})


2019-01-22 19:10:19,455 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10f66eac8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)': /apis/v1beta1/experiments?page_token=&page_size=100&sort_by=
2019-01-22 19:10:19,455 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10f66eac8>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',)': /apis/v1beta1/experiments?page_token=&page_size=100&sort_by=
2019-01-22 19:10:19,455 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10f66eac8>: Failed to establish a new co

MaxRetryError: HTTPConnectionPool(host='ml-pipeline.kubeflow.svc.cluster.local', port=8888): Max retries exceeded with url: /apis/v1beta1/experiments?page_token=&page_size=100&sort_by= (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x10f66ecc0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))